In [1]:
import json
import pathpy as pp

In [2]:
with open('data/language_families.json', 'r') as f:
    families1 = json.loads(f.read())
families = list(families1.items())
families = ('LANGUAGES', families)


FileNotFoundError: [Errno 2] No such file or directory: 'language_families.json'

In [ ]:
indo = ('indo-european', families1['indo-european'])
mongolic = ('mongolic', families1['mongolic'])

In [ ]:
def clean_name(name):
    out = name.replace('\n', '')
    out = out.split('[')[0]
    out = out.split('(')[0]
    out = out.strip()
    return out

In [ ]:
def compat_name(directory, name):
    n_try = name
    n = 2
    if name not in directory:
        return name
    else:
        while n_try in directory:
            n_try = name + str(n)
            n += 1
        return n_try

compat_name(['a', 'b', 'b1'], 'b')

In [ ]:
n = pp.Network()

def walk(tup, depth=0):
    assert len(tup) == 2
    parent = tup[0]
    e1 = clean_name(parent)
    children = tup[1]
    n.add_node(e1, depth=depth)
    for child in children:
        # Names must be new (hence function compat_name)
        # otherwise new node is not added, and subsequent children are
        # appended in the wrong place
        e2_comp = compat_name(n.nodes.keys(), clean_name(child[0]))
        n.add_node(e2_comp, depth=depth, is_final=len(child[1])==0)
        n.add_edge(e1, e2_comp)
        walk((e2_comp, child[1]), depth=depth+1)

walk(indo)

In [ ]:
def walk1(tup, n, depth=0):
    assert len(tup) == 2
    parent = tup[0]
    e1 = clean_name(parent)
    children = tup[1]
    n.add_node(e1, depth=depth)
    for child in children:
        # Names must be new (hence function compat_name)
        # otherwise new node is not added, and subsequent children are
        # appended in the wrong place
        e2_comp = compat_name(n.nodes.keys(), clean_name(child[0]))
        n.add_node(e2_comp, depth=depth, is_final=len(child[1])==0)
        n.add_edge(e1, e2_comp)
        n = walk1((e2_comp, child[1]), n, depth=depth+1)
    
    return n

def network(tup):
    
    return walk1(tup, pp.Network())

fam_list = families[1]
n_list = [network(fam) for fam in fam_list]

In [ ]:
super_n = pp.Network()
for el in n_list:
    super_n += el

In [ ]:
blue = []
red = []

for node, pars in super_n.nodes.items():
    if pars.get('is_final') is True:
        red.append(node)

pp.visualisation.plot(super_n, node_color={el: "#FF6699" for el in red}, force_alpha=0.1)

In [ ]:
pp.visualisation.plot?

In [ ]:
sorted(list(n.nodes.keys()))

In [ ]:
n.nodes['Catalan']

In [ ]:
n.nodes['Unclassified']

In [ ]:
a = pp.Network()
a.add_node('a')
a.add_node('a')
a.nodes

In [ ]:
n.nodes['Catalan']

In [ ]:
n.nodes['indo-european']

In [ ]:
indo[1][0]

In [ ]:
families[1][2][1][0]

In [ ]:

children = families[1]
for child in children:
    for cchild in child[1]:
        print(cchild[0])


In [ ]:
n

In [ ]:
q

In [ ]:
n = pp.Network()
parent = n.add_node('LANGUAGES')
for a, b in families:
    n.add_edge(a, 'LANGUAGES')
    for aa, bb in b:
        n.add_edge(a, aa)
n

In [ ]:
a = dict(a=1, b=2)
list(a.items())

In [ ]:
n = pp.Network()
n.add_node('a')
n.add_edge('a', 'b')
n